In [72]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [73]:
features = pd.read_csv('../Data/training_set_features.csv')
labels = pd.read_csv('../Data/training_set_labels.csv')

In [74]:
df = pd.merge(features, labels, on='respondent_id', how='inner')

In [75]:
df = df.drop(columns=['employment_occupation', 'employment_industry', 'health_insurance', 'respondent_id'])

In [76]:
seas_df = df.drop(columns=['h1n1_concern',
                          'h1n1_knowledge',
                          'doctor_recc_h1n1',
                          'opinion_h1n1_vacc_effective',
                          'opinion_h1n1_risk',
                          'opinion_h1n1_sick_from_vacc',
                          'h1n1_vaccine'])

h1n1_df = df.drop(columns=['doctor_recc_seasonal',
                          'opinion_seas_vacc_effective',
                          'opinion_seas_risk',
                          'opinion_seas_sick_from_vacc',
                          'seasonal_vaccine'])

In [77]:
categorical_columns = [
    'sex',
    'hhs_geo_region',
    'census_msa',
    'race',
    'age_group',
    'behavioral_face_mask',
    'behavioral_wash_hands',
    'behavioral_antiviral_meds',
    'behavioral_outside_home',
    'behavioral_large_gatherings',
    'behavioral_touch_face',
    'behavioral_avoidance',
    'health_worker',
    'child_under_6_months',
    'chronic_med_condition',
    'education',
    'marital_status',
    'employment_status',
    'rent_or_own',
    'doctor_recc_h1n1',
    'doctor_recc_seasonal',
    'income_poverty'
]

numerical_columns = [
    'household_children',
    'household_adults',
    'h1n1_concern',
    'h1n1_knowledge',
    'opinion_h1n1_risk',
    'opinion_h1n1_vacc_effective',
    'opinion_h1n1_sick_from_vacc',
    'opinion_seas_vacc_effective',
    'opinion_seas_risk',
    'opinion_seas_sick_from_vacc',
    
]

for column in categorical_columns:
    curr_col = df[column]
    df.loc[df[column] == 1, column] = 'Yes'
    df.loc[df[column] == 0, column] = 'No'

C:\Users\mcand\Anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


1        False
3        False
4        False
5        False
7        False
         ...  
26699    False
26700    False
26701    False
26703    False
26704    False
Name: behavioral_wash_hands, Length: 22015, dtype: bool

## Initial Run

In [89]:
df = df.dropna()
X_train = df.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])
y_train = df[['h1n1_vaccine', 'seasonal_vaccine']]

#### Categorical

In [90]:
#Get Binary Data for Categorical Variables
cat_df = X_train[categorical_columns]

In [91]:
recat_df = pd.get_dummies(data=cat_df).columns

#### Numerical

In [92]:
num_df = X_train[numerical_columns]

In [93]:
from sklearn.preprocessing import StandardScaler

#Scale Numerical Data
scaler = StandardScaler()
scaled_num = scaler.fit_transform(num_df)
scaled_num_df = pd.DataFrame(scaled_num, index=num_df.index, columns=num_df.columns)

In [98]:
encoded_df = pd.concat([num_df, scaled_num_df], axis=1)

In [99]:
encoded_df

,household_children,household_adults,h1n1_concern,h1n1_knowledge,opinion_h1n1_risk,opinion_h1n1_vacc_effective,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_children,household_adults,h1n1_concern,h1n1_knowledge,opinion_h1n1_risk,opinion_h1n1_vacc_effective,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc
0,0.0,0.0,1.0,0.0,1.0,3.0,2.0,2.0,1.0,2.0,-0.587363,-1.210309,-0.692976,-2.184621,-1.053267,-0.938890,-0.256907,-1.940145,-1.266410,-0.079177
1,0.0,0.0,3.0,2.0,4.0,5.0,4.0,4.0,2.0,4.0,-0.587363,-1.210309,1.558709,1.169714,1.270301,1.097177,1.221174,-0.056950,-0.545672,1.429261
3,0.0,0.0,1.0,1.0,3.0,3.0,5.0,5.0,4.0,1.0,-0.587363,-1.210309,-0.692976,-0.507454,0.495778,-0.938890,1.960215,0.884647,0.895803,-0.833396
4,0.0,1.0,2.0,1.0,3.0,3.0,2.0,3.0,1.0,4.0,-0.587363,0.129688,0.432866,-0.507454,0.495778,-0.938890,-0.256907,-0.998548,-1.266410,1.429261
5,3.0,2.0,3.0,1.0,2.0,5.0,1.0,5.0,4.0,4.0,2.610248,1.469685,1.558709,-0.507454,-0.278745,1.097177,-0.995948,0.884647,0.895803,1.429261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26700,0.0,1.0,3.0,1.0,2.0,4.0,5.0,5.0,4.0,5.0,-0.587363,0.129688,1.558709,-0.507454,-0.278745,0.079144,1.960215,0.884647,0.895803,2.183480
26701,0.0,3.0,2.0,2.0,2.0,4.0,4.0,4.0,2.0,4.0,-0.587363,2.809681,0.432866,1.169714,-0.278745,0.079144,1.221174,-0.056950,-0.545672,1.429261
26702,0.0,0.0,2.0,0.0,1.0,3.0,1.0,5.0,2.0,2.0,-0.587363,-1.210309,0.432866,-2.184621,-1.053267,-0.938890,-0.995948,0.884647,-0.545672,-0.079177
26703,0.0,1.0,1.0,2.0,2.0,4.0,2.0,5.0,1.0,1.0,-0.587363,0.129688,-0.692976,1.169714,-0.278745,0.079144,-0.256907,0.884647,-1.266410,-0.833396
